In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix

import nltk
# import random

In [ ]:
train0=pd.read_csv('/kaggle/input/math80600aw21/train.csv', header=None, names=['label', 'node_id'])
test0=pd.read_csv('/kaggle/input/math80600aw21/test.csv', header=None, names=['node_id']) 
text=pd.read_csv('/kaggle/input/math80600aw21/text.csv', header=None, names=['paper_id','title','abstract'])
nodeid2paperid=pd.read_csv('/kaggle/input/math80600aw21/nodeid2paperid.csv', header=0, names=['node_id','paper_id'])
# sample=pd.read_csv('/content/gdrive/MyDrive/Kaggle/sample.csv', header=0, names= ['node_id','label'])


    Check the Missing Values

In [ ]:
count_nan = len(train0) - train0.count()
count_nan

**preparing the train and test data and saving them
**

merging the train and test data with the text based on column 'node_id'  and frem left side

In [ ]:
text= pd.merge(nodeid2paperid ,text,  how = 'left', on = 'paper_id')
text.head()

In [ ]:
train = pd.merge(train0 ,text,  how = 'left', on = 'node_id')
test = pd.merge(test0 ,text,  how = 'left', on = 'node_id')
print(train.columns)
print(test.columns)

Setting the column order

In [ ]:
train = train[['node_id', 'paper_id', 'title', 'abstract', 'label']]
test = test[['node_id', 'paper_id', 'title', 'abstract']]
print(train.shape)
print(test.shape)
print(text.shape)

Finding maximum length of sentence

In [ ]:
print(train['label'].max())
print(text["title"].map(len).max()) 

Joining 'title' to 'abstract' as one column (named as 'abstract') to use them down the road


In [ ]:
train['abstract'] = train[['title','abstract']].agg('-'.join, axis=1)
test['abstract'] = test[['title','abstract']].agg('-'.join, axis=1)
train.head()

Plotting the label records to see how imbalance the dataset is .
How awfully imbalnced data we have !

In [ ]:
train['label'].value_counts().plot(kind='bar')

In [ ]:
train['label'].value_counts()

Cleaning the text**

In [ ]:
!pip install  unidecode
!pip install word2number 
!pip install  contractions
!pip install  bs4

In [ ]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions

nlp = spacy.load('en')

# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text


def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return " ".join(clean_text)

removing also the mathematical symbols... 

In [ ]:
import re

regex = r"(\$+)(?:(?!\1)[\s\S])*\1"

train['abstract'] =  [re.sub(regex,'', str(x)) for x in train['abstract']]
test['abstract'] =  [re.sub(regex,'', str(x)) for x in test['abstract']]

In [ ]:
train['abstract']=train['abstract'].apply(text_preprocessing)
test['abstract']=test['abstract'].apply(text_preprocessing)

dropping the title since we joined it to 'abstract' column

In [ ]:
train = train.drop('title', axis=1)
test = test.drop('title', axis=1)

Saving the cleaned train and test

In [ ]:
train_clean = pd.DataFrame({'node_id':train.node_id,'paper_id':train.paper_id,'abstract':train.abstract,'label':train.label},columns=['node_id','paper_id','abstract', 'label'])
train_clean.to_csv('/kaggle/input/math80600aw21/train_clean.csv', index=False,header=True)
train_clean.head()

In [ ]:
test_clean = pd.DataFrame({'node_id':test.node_id,'paper_id':test.paper_id,'abstract':test.abstract},columns=['node_id','paper_id','abstract'])
test_clean.to_csv('/kaggle/input/math80600aw21/test_clean.csv', index=False,header=True)
test_clean.head()

**Read prepared tain and test**

In [ ]:
train=pd.read_csv('/kaggle/input/math80600aw21/train_clean.csv')
train.head()

test=pd.read_csv('/kaggle/input/math80600aw21/test_clean.csv')
test.head()

**Text Classification using BiLSTM**

Trying oversampling and Undersampling ( they did not help me )

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# y_train = train['label']
# x_train= train.drop('label', axis=1)
# # define undersample strategy
# undersample = RandomUnderSampler() 
# # fit and apply the transform
# X_under, Y_under = undersample.fit_resample(x_train, y_train)

# df1 = pd.DataFrame(X_under,columns=['abstract'])
# df2 = pd.DataFrame(Y_under,columns=['label'])
# df = df1.join(df2)
# # df = pd.DataFrame(df,columns=['abstract','label'])
# # df=df.groupby('label').apply(lambda x: x.sample(1000))
# print(df.shape)
# df['label'].value_counts().plot(kind='bar')
# X_under=df.drop('label', axis=1)
# Y_under =df['label']

In [ ]:
# train = train.drop(train[train['label'] == 16].sample(frac=0.7).index)
# train = train.drop(train[train['label'] == 10].sample(frac=0.3).index)
# train = train.drop(train[train['label'] == 8].sample(frac=0.2).index)
# train = train.drop(train[train['label'] == 4].sample(frac=0.2).index)
# train = train.drop(train[train['label'] == 5].sample(frac=0.1).index)
# train = train.drop(train[train['label'] == 2].sample(frac=0.1).index)


# train['label'].value_counts()
# train['label'].value_counts().plot(kind='bar')

In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers 
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

from tensorflow import keras
from keras.models import Sequential

See the important words in each category 

In [ ]:
from nltk.corpus import stopwords# nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [ ]:
from wordcloud import WordCloud
n_posts = 5000
w_0 = ' '.join(train[train['label'] == 0]['abstract'].str.lower().values[:n_posts])
w_1 = ' '.join(train[train['label'] == 1]['abstract'].str.lower().values[:n_posts])
w_2 = ' '.join(train[train['label'] == 2]['abstract'].str.lower().values[:n_posts])
w_3 = ' '.join(train[train['label'] == 3]['abstract'].str.lower().values[:n_posts])


wordcloud_0 = WordCloud(max_font_size=None, stopwords=stop,scale = 2,colormap = 'Dark2').generate(w_0)
wordcloud_1 = WordCloud(max_font_size=None, stopwords=stop,scale = 2,colormap = 'Dark2').generate(w_1)
wordcloud_2 = WordCloud(max_font_size=None, stopwords=stop,scale = 2,colormap = 'Dark2').generate(w_2)
wordcloud_3 = WordCloud(max_font_size=None, stopwords=stop,scale = 2,colormap = 'Dark2').generate(w_3)

fig, ax = plt.subplots(1,4, figsize=(20, 5))
ax[0].imshow(wordcloud_0)
ax[0].set_title('Top words in label 1',fontsize = 20)
ax[0].axis("off")

ax[1].imshow(wordcloud_1)
ax[1].set_title('Top words in label 2',fontsize = 20)
ax[1].axis("off")

ax[2].imshow(wordcloud_2)
ax[2].set_title('Top words in label 1',fontsize = 20)
ax[2].axis("off")

ax[3].imshow(wordcloud_3)
ax[3].set_title('Top words in label 2',fontsize = 20)
ax[3].axis("off")


plt.show()

In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Reading glove embedding and trying different dimensions : 50,100,200 

In [ ]:
EMBEDDING_FILE= '../input/glove6b50dtxt/glove.6B.200d.txt'


In [ ]:
embed_size = 200 # length of each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a abstract

Tokenizing the senetences 

In [ ]:
list_sentences_train = train['abstract'].values
list_sentences_test = test['abstract'].values

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [ ]:
embedding_matrix.shape

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))+1
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
inp = Input(shape=(maxlen,))
inp.shape

defining the model

In [ ]:
inp = Input(shape=(maxlen,))

x = Embedding(20001, embed_size, weights=[embedding_matrix],trainable=False)(inp)
x = Bidirectional(LSTM(512, return_sequences=True, dropout=0.3, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(400, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(200, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(20, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

using early stopping to avoid overfitting. also saving the best model in terms of accuracy on valid set

In [ ]:
# model = getModel()
model.summary()


earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=0, mode='max')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy',mode='max',)


trying to give weights to classes ( did not help me much) 

In [ ]:
# from sklearn.utils import class_weight
# class_weights = class_weight.compute_class_weight('balanced', np.unique(y),y)
# class_weights

# weight_dict={}
# for i in range(len(class_weights)):
#   weight_dict[i] = class_weights[i]
# weight_dict


categorzing the target variable to use it in the model/ 

In [ ]:
y = train['label'].values
np.unique(y)
from keras.utils import to_categorical
y = to_categorical(y, 20)
y

In [ ]:
model.fit(X_t, y, batch_size=1024, epochs=100, verbose=1, callbacks=[earlyStopping, mcp_save] , validation_split=0.1)  #class_weight=weight_dict

In [ ]:
# model.save('/content/gdrive/MyDrive/Kaggle')
# from tensorflow import keras
# model = keras.models.load_model('/content/gdrive/MyDrive/Kaggle')
#!nvidia-smi

f1 metric since we are doing multi class classification and we have an imbalanced dataset to see the performance in each class as well

In [ ]:
from sklearn.metrics import f1_score
y_pred = model.predict([X_t], batch_size=1024, verbose=1)
y_pred=np.argmax(y_pred,axis=1)
y_true=train['label']


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, labels=[0,1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]))

In [ ]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)

In [ ]:
predict = pd.DataFrame({'id':test.node_id,'label':pred},columns=['id', 'label'])
predict.to_csv('/kaggle/input/math80600aw21/predict1.csv', index=False,header=True)
predict.shape

I have tested different 
-activation functions : tanh , elu , relu, leakyrelu, ... 
-number of neurons for diferent BiLSTM and dense layers : from 32-512 and  from 20-400  respectivly
-Batch sizes : from 8-4096 and
-Learning rates : 1e-1 to 1e-5
-also used dropout and BatchNormalisation. 
-I have also tried ubdersampling,oversampling and giving weights to the classes to address data imbalance. 
and ... 

I could not get better resulst than 79% accuarcy with BiLSTM

**Using the Bert model**

Matching the versions in differnt packages

In [ ]:
!pip install 'torch== 1.4.0'
!pip install 'pandas== 1.0.3'
!pip install 'numpy ==1.18.2'
!pip install 'transformers==2.8.0'

In [ ]:
train=pd.read_csv('/kaggle/input/math80600aw21/train_clean.csv')
train

In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch.nn import functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

using the "cased" version of Bert since my text is not like a tweet, so nobody writes BAD in an academic text instead of bad. o there is no semantic difference in capital letters or Not-capital letters.  

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'

loading a pre-trained BertTokenizer:

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Choosing Sequence Length BERT works with fixed-length sequences. We’ll use a simple strategy to choose the max length. Let’s store the token length of each review:

In [ ]:
df=train.copy()

token_lens = []
for txt in df.abstract:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

Most of the reviews seem to contain less than 250 tokens, but we’ll be on the safe side and choose a maximum length of 250.

In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 512]);
plt.xlabel('Token count');

In [ ]:
MAX_LEN = 250

In [ ]:
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.reviews)
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)
df_train.shape, df_val.shape, df_test.shape

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.abstract.to_numpy(),
    targets=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )
BATCH_SIZE = 15
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
data = next(iter(train_data_loader))
data.keys()


print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

Check our training data loader:

In [ ]:
data = next(iter(train_data_loader))
data.keys()


print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
last_hidden_state, pooled_output = bert_model(input_ids=encoding['input_ids'], attention_mask=encoding['attention_mask'])

768  is the number of hidden units in the feedforward-networks. We can verify that by checking the config:

In [ ]:
bert_model.config.hidden_size

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
model = SentimentClassifier(len(df['label']))
model = model.to(device)

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

In [ ]:
attention_mask

In [ ]:
EPOCHS = 5

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=True)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

In [ ]:
test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

In [ ]:
print(classification_report(y_test, y_pred)) #, target_names=class_names

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/Kaggle/slow_model_1.pt')
# slow_model = torch.load('/content/gdrive/MyDrive/Kaggle/slow_model.pt')

In [ ]:
test['label']=None

In [ ]:
for i in range(len(test)):
 
  encoded_review = tokenizer.encode_plus(test['abstract'][i], 
                                         max_length=MAX_LEN,
                                         add_special_tokens=True, 
                                         return_token_type_ids=False,  
                                         pad_to_max_length=True,  
                                         return_attention_mask=True,  
                                         return_tensors='pt',)
  
  input_ids = encoded_review['input_ids'].to(device)
  attention_mask = encoded_review['attention_mask'].to(device)
  
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  test['label'][i]=prediction.item()
  

In [ ]:
prediction_bert = pd.DataFrame({'node_id':test.node_id,'paper_id':test.paper_id,'abstract':test.abstract,'label':test.label},columns=['node_id','paper_id','abstract','label'])

prediction_bert = prediction_bert.drop('paper_id', axis=1)
prediction_bert = prediction_bert.drop('abstract', axis=1)

prediction_bert = pd.DataFrame({'id':prediction_bert.node_id,'label':prediction_bert.label},columns=['id', 'label'])
prediction_bert.to_csv('/content/gdrive/MyDrive/Kaggle/prediction_bert_1.csv', index=False, header=True)
prediction_bert.shape

**Using two (Pre-trained and Not Pre-Trained Bert( different from the previous OFF-THE_SHELF Bert model that I have used from HuggingFace Tutorials) models and averaging their results as final prediction**

I have used differnt number of layers and hyperparamters besied using some other pre-trained models like Roberta but Bert has yielded better results for me. 

**Using pretrained and not-pretrained BERT (ensemble)**

Checking the Colab 

In [ ]:
!nvidia-smi

In cacse, If they are not installed 

In [ ]:
# !pip install Spacy
! pip install transformers > null
! pip install tokenizers > null

In [ ]:
import torch
from torch.nn import functional as F
import pandas as pd
import gc
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.empty_cache()
import numpy as np
import gc
gc.collect()
import pandas as pd
import transformers
from transformers import AutoModel,AutoConfig
import plotly
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from sklearn.cluster import KMeans
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import scipy
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, accuracy_score,f1_score
import time
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

batch_size = 32
max_len = 250
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Kaggle/train_clean.csv")
print(len(train_data))
# valid_data = pd.read_csv("/content/drive/MyDrive/Kaggle/test_clean.csv")
valid_data = train_data.sample(frac=0.1)
train_data = train_data.drop(index=valid_data.index)
print(len(train_data))
print(len(valid_data))

double check

In [ ]:
train_data["abstract"] = train_data["abstract"].astype(str)
valid_data["abstract"] = valid_data["abstract"].astype(str)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df,tokenizer,max_len=32):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        df = self.df.iloc[idx]
        if isinstance(idx, int):
            abstract = [df['abstract']]
            label = [df['label']]
            
        else:
            abstract = df['abstract'].to_list()
            label = df['label'].to_list()
            
        abstract_t = self.tokenizer(abstract, padding="max_length", truncation=True, return_tensors="pt", max_length=self.max_len)
        label_t = torch.LongTensor(label)
        
        return {'input_ids':abstract_t['input_ids'].squeeze().to(device),
                "attention_mask":abstract_t['attention_mask'].squeeze().to(device),
                "token_type_ids":abstract_t['token_type_ids'].squeeze().to(device),
                "label":label_t.squeeze().to(device),
               
                   }

In [ ]:
# from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', return_dict=True)
# model.to(device)

# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# from transformers import AdamW
# # optimizer = AdamW(model.parameters(), lr=1e-5)

calling both Pre-trained and NOt-pretrained Bert

In [ ]:
from transformers import AutoModelForSequenceClassification
# model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', return_dict=True)
# model.to(device)

from transformers import AutoConfig
config = AutoConfig.from_pretrained('bert-base-uncased')
model =  AutoModelForSequenceClassification.from_config(config)
model.to(device)


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

from transformers import AdamW
# optimizer = AdamW(model.parameters(), lr=1e-5)


In [ ]:
train_dataset = MyDataset(train_data, tokenizer, max_len=250)
valid_dataset = MyDataset(valid_data, tokenizer, max_len=250)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

I could have also used 1024 hidden units for Bert but I used  768 to be easier since my computational power was low.

In [ ]:
from torch.autograd import Function
class MainModule(nn.Module):
    def __init__(self, bert):
        super(MainModule, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.25)
        self.cls_out = nn.Linear(768, 20)
        
    def forward(self,batch_data):
        shared_output = self.bert(batch_data['input_ids'], batch_data['attention_mask'], batch_data['token_type_ids'])  
        ###################################      
        pooled_output = shared_output[1]
        ###################################
        pooled_output_dropped = self.dropout(pooled_output)
        logits_out = self.cls_out(pooled_output_dropped)
        return_dict = {"logits_out":logits_out}
        return return_dict

In [ ]:
# from torch.autograd import Function
# class MainModule(nn.Module):
#     def __init__(self, bert):
#         super(MainModule, self).__init__()
#         self.bert = bert
#         self.dropout = nn.Dropout(0.5)
#         self.classifier = nn.Sequential(
#             nn.Linear(768, 500),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(500, 20),
#             nn.Dropout(0.5)
#         )

#     def forward(self,batch_data):
#         shared_output = self.bert(batch_data['input_ids'], batch_data['attention_mask'], batch_data['token_type_ids'])  
#         ###################################      
#         pooled_output = shared_output[1]
#         ###################################
#         pooled_output_dropped = self.dropout(pooled_output)
#         logits_out = self.classifier(pooled_output_dropped)
#         return_dict = {"logits_out":logits_out}
#         return return_dict

In [ ]:
model = MainModule(model.bert)
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
colors=['blue','green','cyan', 'black', 'red','yellow','brown']
def plot_cm(labels, preds):
    cm = confusion_matrix(labels, preds)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    fig.colorbar(cax)
    for i in range(len(cm)):
        for j in range(len(cm)):
            plt.annotate(cm[i,j],xy=(j,i),horizontalalignment='center',verticalalignment='center',size=15,color='orange') 
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
def report(logits, labels):
    logits = np.concatenate(logits, axis=0)
    logits = np.argmax(logits, axis=-1)
    labels = np.concatenate(labels, axis=0)
    print(classification_report(labels, logits))
    plot_cm(labels, logits)

In [ ]:
def model_evaluation(model, data_loader):
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    with torch.no_grad():
        labels = []
        logits = []
        
        for batch, batch_data in enumerate(data_loader):
            # optimizer.zero_grad()
            outputs = model(batch_data)
            labels.append(batch_data['label'].cpu().detach().numpy())
            logits.append(outputs['logits_out'].cpu().detach().numpy())
        # loss /= (batch + 1)
        report(logits, labels)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

model.train()
epochs = 1
for epoch in range(1,epochs+1):
    labels_out_list = []
    logits_out_list = []
    
    for batch, batch_data in enumerate(train_data_loader):
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = F.cross_entropy(outputs['logits_out'], batch_data['label'])
        
        loss.backward()
        optimizer.step()

        labels_out_list.append(batch_data['label'])
        logits_out_list.append(outputs['logits_out'])
       


        log_interval = 25
        if batch % log_interval == 0:
            print(" epoch=",epoch,"  batch=",batch,"   loss=",loss.item())
    print('------------------------------------')
    labels_out_list = [i.cpu().detach().numpy() for i in labels_out_list]
    logits_out_list = [i.cpu().detach().numpy() for i in logits_out_list]
    report(logits_out_list, labels_out_list)
    print('************************')
    model_evaluation(model, valid_data_loader)
    # torch.save(model, "/content/drive/MyDrive/Kaggle/models/" + str(epoch) +".pt")

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Kaggle/test_clean.csv')
test

In [ ]:
test['label']=None
# test = test.drop('paper_id', axis=1)

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, df,tokenizer,max_len=32):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        df = self.df.iloc[idx]
        if isinstance(idx, int):
            abstract = [df['abstract']]
            
        else:
            abstract = df['abstract'].to_list()
            
        abstract_t = self.tokenizer(abstract, padding="max_length", truncation=True, return_tensors="pt", max_length=self.max_len)
        
        return {'input_ids':abstract_t['input_ids'].squeeze().to(device),
                "attention_mask":abstract_t['attention_mask'].squeeze().to(device),
                "token_type_ids":abstract_t['token_type_ids'].squeeze().to(device),
                }

In [ ]:
test_dataset = MyDatasetTest(test, tokenizer, max_len=250)


test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
def model_prediction(model, data_loader):
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    with torch.no_grad():
        
        logits = []
        
        for batch, batch_data in enumerate(data_loader):
            
            outputs = model(batch_data)
            
            logits.append(outputs['logits_out'].cpu().detach().numpy())
        
    return logits

I saved the models to be able to use them again 

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/Kaggle/model_bert_not_trained.pt')
torch.save(model, "/content/drive/MyDrive/Kaggle/Model_whole_bert_not_trained.pt")
# Model class must be defined somewhere
# model = torch.load("/content/drive/MyDrive/Kaggle/Model_whole_bert.pt")
# model.eval()

In [ ]:
model_pretrained = torch.load("/content/drive/MyDrive/Kaggle/Model_whole_bert.pt")

In [ ]:
labels_bert_pretrained=model_prediction(model_pretrained,test_data_loader)
labels_bert_pretrained

In [ ]:
logits_pretrained = np.concatenate(labels_bert_pretrained, axis=0)
logits_pretrained.shape

using Softmax to get pred probs

In [ ]:
from tqdm import tqdm
labels_bert_pretrained_prob = []
for i in tqdm(logits_pretrained):
  sum = np.sum(np.exp(i))
  labels_bert_pretrained_prob.append(np.exp(i)/sum)
labels_bert_pretrained_prob

In [ ]:
model_not_trained = torch.load("/content/drive/MyDrive/Kaggle/Model_whole_bert_not_trained.pt")

In [ ]:
labels_bert_not_pretrained=model_prediction(model_not_trained,test_data_loader)
labels_bert_not_pretrained

In [ ]:
logits_not_pretrained = np.concatenate(labels_bert_not_pretrained, axis=0)
logits_not_pretrained.shape

using Softmax to get pred probs

In [ ]:
labels_bert_not_pretrained_prob = []
for i in tqdm(logits_not_pretrained):
  sum = np.sum(np.exp(i))
  labels_bert_not_pretrained_prob.append(np.exp(i)/sum)
labels_bert_not_pretrained_prob

Averaging the results

In [ ]:
labels = []
for i in range(len(labels_bert_not_pretrained_prob)):
  labels.append((labels_bert_not_pretrained_prob[i] + labels_bert_pretrained_prob[i])/2)
# labels = (labels_bert_not_pretrained_prob + labels_bert_pretrained_prob) / 2
labels

In [ ]:
test.to_csv('/content/drive/MyDrive/Kaggle/submit.csv', index=False)

In [ ]:
# logits = np.concatenate(labels, axis=0)
logits = np.argmax(labels, axis=-1)

In [ ]:
test['label']=logits
test

In [ ]:
prediction_Java = pd.DataFrame({'id':test.node_id,'label':test.label},columns=['id','label'])



In [ ]:
prediction_Java.to_csv('/content/drive/MyDrive/Kaggle/prediction_Java.csv', index=False, header=True)
prediction_Java.shape

In [ ]:
prediction_Java